In [1]:
# 1. 코랩 환경에 필요한 라이브러리 설치
# gymnasium (RL 환경), pygame (렌더링 엔진), moviepy (비디오 저장)
!pip install gymnasium pygame moviepy

In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame
import random
import os
import glob
import io
import base64
import datetime
from collections import deque
from IPython.display import HTML, display
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from google.colab import drive
import shutil

# =========================================================
# 1. 구글 드라이브 마운트 (저장소 연결)
# =========================================================
drive.mount('/content/drive')
GDRIVE_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/reinforcement learning/Final_Model_4to8'

# 폴더가 없으면 새로 만듭니다. (에러 방지)
if not os.path.exists(GDRIVE_BASE_PATH):
    os.makedirs(GDRIVE_BASE_PATH)

# =========================================================
# 2. 헬퍼 함수 정의
# =========================================================

def show_video(video_path, video_width=600):
    """
    [비디오 재생기]
    저장된 mp4 파일을 읽어서 코랩 화면에 플레이어로 띄워줍니다.
    """
    if not os.path.exists(video_path): return
    video_file = open(video_path, "rb").read()
    # 비디오 데이터를 웹 브라우저가 이해할 수 있는 문자열(base64)로 변환
    video_url = f"data:video/mp4;base64,{base64.b64encode(video_file).decode()}"
    # HTML 태그를 이용해 화면에 출력
    display(HTML(f"""<video width="{video_width}" controls autoplay loop><source src="{video_url}"></video>"""))

def get_success_threshold(grid_size):
    """
    [합격 커트라인 계산]
    목표 점수 = 잭팟 보너스(500) + 전체 타일 수(칸당 1점 보너스 감안)
    이 점수를 넘어야 '완벽하게 클리어했다'고 판단합니다.
    """
    total_cells = grid_size * grid_size
    return 500.0 + float(total_cells)

def evaluate_and_record(agent, max_grid_size, current_grid_size, episode_num, run_id):
    """
    [시각적 평가 - CCTV 녹화]
    현재 에이전트의 실력을 눈으로 확인하기 위해 한 판을 녹화합니다.
    """
    video_dir = f"./videos/{run_id}/eval_{episode_num}"

    # 평가용 환경 생성 (rgb_array 모드는 화면을 이미지로 뱉어냄 -> 녹화 가능)
    eval_env = ZipGeneralEnv(
        max_grid_size=max_grid_size,
        min_grid_size=current_grid_size,
        target_grid_size=current_grid_size,
        render_mode="rgb_array"
    )
    # 난이도를 고정합니다 (예: 5x5면 5x5만 나오게)
    eval_env.set_curriculum_level(current_grid_size)

    # Gym의 녹화 기능(Wrapper)을 환경에 씌웁니다.
    env_to_record = gym.wrappers.RecordVideo(eval_env, video_dir, episode_trigger=lambda e: e == 0)

    # 게임 시작!
    obs, info = env_to_record.reset()
    valid_actions = info['valid_actions'] # 첫 번째 유효 행동 수신
    done = False; total_reward = 0

    # 평가 중에는 탐험(랜덤 행동)을 끕니다.
    orig_eps = agent.epsilon
    agent.epsilon = 0.0

    while not done:
        # 에이전트가 행동 결정
        action = agent.act(obs, valid_actions)

        # 행동 실행
        obs, reward, terminated, truncated, info = env_to_record.step(action)

        # [중요] 바뀐 상황에서의 유효 행동 업데이트
        valid_actions = info['valid_actions']

        done = terminated or truncated
        total_reward += reward

    # 녹화 종료 및 자원 해제
    env_to_record.close()

    # 탐험률을 원래대로 돌려놓습니다 (훈련 재개를 위해)
    agent.epsilon = orig_eps

    print(f"   >>> 🎥 [Video Rec] Size:{current_grid_size}x{current_grid_size} | Score:{total_reward:.1f}")

    # 저장된 비디오 파일 찾아서 화면에 보여주기
    video_files = glob.glob(f"{video_dir}/*.mp4")
    if video_files:
        latest = max(video_files, key=os.path.getmtime) # 가장 최근 파일
        show_video(latest, video_width=300)

    return total_reward

def run_comprehensive_exam(agent, eval_env_instance, size_list, episodes_per_size=10):
    """
    [종합 졸업 시험]
    주어진 사이즈 목록(예: 4, 6, 8)에 대해 각각 10번씩 시험을 봅니다.
    모든 사이즈에서 평균 90% 이상의 승률을 보여야 '졸업'입니다.
    """
    print(f"\n   👮 [종합 졸업 시험] 범위 {size_list} 검증 시작 (각 {episodes_per_size}판)...")

    orig_eps = agent.epsilon
    agent.epsilon = 0.0 # 커닝 금지 (순수 실력)

    results = {}
    all_passed = True # 일단 합격이라고 가정하고 검증 시작

    for size in size_list:
        # 시험 환경 난이도 설정
        eval_env_instance.set_curriculum_level(size)
        target_score = get_success_threshold(size) # 합격 기준 점수
        success_count = 0

        # 정해진 횟수만큼 반복 시험
        for _ in range(episodes_per_size):
            obs, info = eval_env_instance.reset()
            valid_actions = info['valid_actions']
            done = False; total_reward = 0

            while not done:
                action = agent.act(obs, valid_actions)
                obs, reward, terminated, truncated, info = eval_env_instance.step(action)
                valid_actions = info['valid_actions']
                done = terminated or truncated
                total_reward += reward

            # 기준 점수 넘으면 성공 카운트
            if total_reward >= target_score:
                success_count += 1

        # 성공률 계산
        pass_rate = (success_count / episodes_per_size) * 100
        results[size] = pass_rate
        print(f"      - {size}x{size}: 성공률 {pass_rate:.0f}%")

        # 90점 미만인 과목이 하나라도 있으면 졸업 실패
        if pass_rate < 90.0:
            all_passed = False

    # 시험 끝났으니 원래대로 복구
    agent.epsilon = orig_eps
    return all_passed, results

Mounted at /content/drive


In [4]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame
import random
import os
from collections import deque # BFS 탐색을 위한 효율적인 큐(Queue)

class ZipGeneralEnv(gym.Env):
    """
    [Zip 게임 강화학습 환경 - Dual View & Dead-end 감지 최종판]

    이 환경은 에이전트에게 두 가지 시각 정보와 강력한 피드백을 제공합니다.

    1. 시각 정보 (Dual Eyes):
       - Local View (7x7): 내 주변의 미세한 장애물을 피하기 위함 (전술)
       - Global View (12x12): 맵 전체의 연결 상태와 고립을 파악하기 위함 (전략)

    2. 핵심 로직 (Dead-end Detection):
       - BFS 알고리즘을 사용하여 에이전트가 이동할 때마다 미래를 시뮬레이션합니다.
       - '더 이상 깰 수 없는 상태'가 감지되면 즉시 게임을 종료(Kill)하고 큰 벌점을 줍니다.
    """
    metadata = {'render_modes': ['human', 'rgb_array'], 'render_fps': 10}

    def __init__(self, max_grid_size=12, min_grid_size=4, target_grid_size=8, render_mode=None):
        super().__init__()

        # ---------------------------------------------------------
        # 1. 환경 기본 설정
        # ---------------------------------------------------------
        self.max_grid_size = max_grid_size   # 메모리 할당용 최대 크기
        self.min_grid_size = min_grid_size   # 시작 난이도
        self.target_grid_size = target_grid_size
        self.render_mode = render_mode
        self.grid_size = min_grid_size

        self.cell_size = 40
        self.window_size = self.max_grid_size * self.cell_size
        self.screen = None; self.clock = None; self.font = None

        # ---------------------------------------------------------
        # 2. 시야 설정 (Local View용)
        # ---------------------------------------------------------
        self.view_range = 7
        self.view_pad = self.view_range // 2

        # ---------------------------------------------------------
        # 3. 데이터 규격 (Observation Space) - [수정됨: 3개 입력]
        # ---------------------------------------------------------
        self.observation_space = spaces.Dict({
            # (A) Local: 내 주변 7x7 (미세 컨트롤용)
            "local": spaces.Box(low=0, high=self.max_grid_size**2,
                               shape=(self.view_range, self.view_range, 4), dtype=np.float32),

            # (B) Global: 맵 전체 12x12 (전체 전략용) - 맵이 작으면 나머지는 0으로 패딩됨
            "global": spaces.Box(low=0, high=self.max_grid_size**2,
                               shape=(self.max_grid_size, self.max_grid_size, 4), dtype=np.float32),

            # (C) Vector: 방향 및 상태 수치 정보
            "vector": spaces.Box(low=-1.0, high=1.0, shape=(9,), dtype=np.float32)
        })

        self.action_space = spaces.Discrete(4) # 상, 하, 좌, 우
        self.nodes = {} # 숫자 노드 위치

        # 난이도 분포 (Main Loop에서 제어)
        self.size_distribution = {4: 1.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0}

        # 4. Pygame 초기화
        if self.render_mode == 'rgb_array':
            os.environ["SDL_VIDEODRIVER"] = "dummy"
        if self.render_mode in ['human', 'rgb_array']:
            self._init_pygame()

    def _init_pygame(self):
        """화면 그리기 도구 준비"""
        pygame.init()
        self.screen = pygame.Surface((self.window_size, self.window_size))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 24)
        if self.render_mode == 'human':
            pygame.display.set_caption("Zip General (Dual View)")
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))

    def set_curriculum_level(self, new_size):
        """평가용: 특정 난이도만 100% 나오게 고정"""
        if new_size <= self.max_grid_size:
            self.grid_size = new_size
            self.min_grid_size = new_size
            self.target_grid_size = new_size
            self.max_episode_steps = self.grid_size * self.grid_size * 3
            self.total_cells = self.grid_size * self.grid_size
            self.size_distribution = {s: (1.0 if s == new_size else 0.0) for s in range(4, 9)}

    def reset(self, seed=None, options=None):
        """[새 게임] 맵을 생성하고 초기화"""
        super().reset(seed=seed)

        # 1. 확률 분포에 따라 맵 크기 결정
        sizes = list(self.size_distribution.keys())
        weights = list(self.size_distribution.values())
        if sum(weights) > 0:
            self.grid_size = random.choices(sizes, weights=weights, k=1)[0]
        else:
            self.grid_size = self.min_grid_size

        self.max_episode_steps = self.grid_size * self.grid_size * 3
        self.total_cells = self.grid_size * self.grid_size

        # 2. 맵 생성 (정답 경로 포함)
        min_nodes = 3
        max_nodes = min(self.grid_size + 2, 20)
        self.num_nodes = random.randint(min_nodes, max_nodes)
        self._generate_solvable_map()

        # 3. 상태 초기화
        self.current_target = 2
        self.path_drawn = set()
        self.path_drawn.add(self.pen_pos)
        self.current_step = 0

        if self.render_mode == 'human': self._render_frame()
        return self._get_observation(), self._get_info()

    # ⭐️ [핵심 알고리즘] 경로 소멸 감지 (BFS)
    # 역할: "만약 내가 거기로 가면(Simulate), 게임을 깰 수 있는가?"를 판단
    def _is_dead_end(self, next_pos, visited_set):
        # 1. 가상 방문 처리 (시뮬레이션)
        temp_visited = visited_set.copy()
        temp_visited.add(next_pos)

        target_pos = self.nodes.get(self.current_target)

        # 2. 남은 빈칸 개수 확인
        # (도달해야 할 총 칸 수)
        total_remaining_cells = 0
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if (r, c) not in temp_visited:
                    total_remaining_cells += 1

        # 빈칸이 0개면 성공 직전이므로 Dead End 아님
        if total_remaining_cells == 0: return False

        # 3. [검증 1] 목표 도달 가능성 확인
        # 규칙: "지금 당장 목표 숫자까지 가는 길이 뚫려있는가?"
        # 주의: 미래의 숫자 노드(다음 목표들)는 현재 '벽'으로 취급됨
        queue = deque([next_pos])
        bfs_visited = temp_visited.copy()
        target_found = False

        while queue:
            r, c = queue.popleft()
            if (r, c) == target_pos:
                target_found = True; break

            for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                nr, nc = r + dr, c + dc
                if (0 <= nr < self.grid_size and 0 <= nc < self.grid_size):
                    if (nr, nc) not in bfs_visited:
                        # 숫자 노드는 순서가 아니면 통과 불가 (벽)
                        if (nr, nc) in self.nodes.values() and (nr, nc) != target_pos:
                            continue
                        bfs_visited.add((nr, nc))
                        queue.append((nr, nc))

        # 목표 자체가 막혔으면 -> 사망
        if not target_found and target_pos is not None: return True

        # 4. [검증 2] 전체 연결성 확인 (고립 감지)
        # 규칙: "미래에 모든 숫자가 열린다고 가정했을 때, 못 가는 빈칸이 생기는가?"

        # 남은 빈칸 중 아무거나 하나 잡아서 시작점으로 설정
        start_node = None
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if (r, c) not in temp_visited:
                    start_node = (r, c); break
            if start_node: break

        if not start_node: return False # 빈칸 없음

        # 완화된 BFS (숫자 노드 통과 가능)
        queue = deque([start_node])
        bfs_visited_relaxed = temp_visited.copy()
        reachable_count = 0

        while queue:
            r, c = queue.popleft()
            reachable_count += 1

            for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                nr, nc = r + dr, c + dc
                if (0 <= nr < self.grid_size and 0 <= nc < self.grid_size):
                    if (nr, nc) not in bfs_visited_relaxed:
                        bfs_visited_relaxed.add((nr, nc))
                        queue.append((nr, nc))

        # 내가 갈 수 있는 칸 < 전체 남은 칸 -> 어딘가 고립됨 -> 사망
        if reachable_count < total_remaining_cells: return True

        return False # 두 검증 통과 시 안전

    def step(self, action):
        self.current_step += 1
        (r, c) = self.pen_pos

        # 1. 이동 좌표 계산
        if action == 0: new_pos = (r - 1, c)
        elif action == 1: new_pos = (r + 1, c)
        elif action == 2: new_pos = (r, c - 1)
        elif action == 3: new_pos = (r, c + 1)

        # 맵 밖 처리
        if not (0 <= new_pos[0] < self.grid_size and 0 <= new_pos[1] < self.grid_size):
            new_pos = self.pen_pos

        self.pen_pos = new_pos
        is_new_tile = new_pos not in self.path_drawn

        reward = 0.0; terminated = False; truncated = False
        reward -= 0.1 # 시간 비용

        if is_new_tile:
            # ⭐️ [핵심] 조기 종료 및 강력한 벌점
            if self._is_dead_end(new_pos, self.path_drawn):
                # 벌점: 남은 빈칸 개수만큼 추가 벌점 부여 (초반 실수일수록 더 크게 혼남)
                visited_count = len(self.path_drawn) + 1
                remaining_cells = self.valid_cells_count - visited_count
                penalty = 50.0 + float(remaining_cells)

                reward -= penalty
                terminated = True # 즉시 게임 오버

                # 마지막 위치 반영
                self.pen_pos = new_pos
                self.path_drawn.add(new_pos)
                return self._get_observation(), reward, terminated, truncated, self._get_info()
            else:
                reward += 2.0 # 생존 보상

            self.pen_pos = new_pos
            self.path_drawn.add(new_pos)
        else:
            self.pen_pos = new_pos

        # 2. 목표 도달 체크
        current_target_pos = self.nodes.get(self.current_target)
        if self.pen_pos == current_target_pos:
            if self.current_target == self.actual_num_nodes:
                # 최종 완료
                missed = self.valid_cells_count - len(self.path_drawn)
                if missed == 0:
                    reward += 500.0; terminated = True # 잭팟
                else:
                    # (이론상 도달 못하지만 안전장치)
                    clear_bonus = 200.0 - (float(missed) * 5.0)
                    reward += max(0.0, clear_bonus); terminated = True
            else:
                # 중간 목표
                reward += 10.0; self.current_target += 1

        # 3. 갇힘 체크 (BFS 덕분에 거의 발생 안 함)
        info = self._get_info()
        if not terminated and info['is_stuck']:
            terminated = True
            missed = self.valid_cells_count - len(self.path_drawn)
            reward -= (50.0 + float(missed))

        # 4. 시간 초과
        if self.current_step >= self.max_episode_steps: truncated = True

        return self._get_observation(), reward, terminated, truncated, info

    def _get_observation(self):
        """
        [AI의 눈 - Dual View 생성]
        1. Local View (7x7): 미세 컨트롤용
        2. Global View (12x12): 전역 전략용
        """
        shape = (self.max_grid_size, self.max_grid_size)

        # 4개 채널 정보 생성
        node_board = np.zeros(shape, dtype=np.float32)
        path_board = np.zeros(shape, dtype=np.float32)
        pen_pos_board = np.zeros(shape, dtype=np.float32)
        target_board = np.zeros(shape, dtype=np.float32)

        for num, (r, c) in self.nodes.items(): node_board[r, c] = num
        for (r, c) in self.path_drawn: path_board[r, c] = 1.0
        pen_pos_board[self.pen_pos] = 1.0
        t_pos = self.nodes.get(self.current_target, (-1, -1))
        if t_pos != (-1, -1): target_board[t_pos[0], t_pos[1]] = 1.0

        # ⭐️ [Global View] 전체 맵 스택
        global_grid_obs = np.stack([
            node_board, path_board, pen_pos_board, target_board
        ], axis=-1)

        # ⭐️ [Local View] 7x7 크롭 & 패딩
        pad = self.view_pad
        padded_node = np.pad(node_board, pad, constant_values=-1)
        padded_path = np.pad(path_board, pad, constant_values=1)
        padded_pen = np.pad(pen_pos_board, pad, constant_values=0)
        padded_target = np.pad(target_board, pad, constant_values=0)

        pr, pc = self.pen_pos[0] + pad, self.pen_pos[1] + pad
        slice_r = slice(pr - pad, pr + pad + 1)
        slice_c = slice(pc - pad, pc + pad + 1)

        local_grid_obs = np.stack([
            padded_node[slice_r, slice_c], padded_path[slice_r, slice_c],
            padded_pen[slice_r, slice_c], padded_target[slice_r, slice_c]
        ], axis=-1)

        # [Vector] 수치 정보
        norm = float(self.max_grid_size)
        scale_ratio = self.grid_size / self.max_grid_size
        cy, cx = self.pen_pos
        if t_pos != (-1, -1):
            ty, tx = t_pos
            dy, dx = (ty - cy) / norm, (tx - cx) / norm
        else: dy, dx = 0, 0
        progress_ratio = (self.current_target - 1) / max(self.actual_num_nodes, 1)
        fill_ratio = len(self.path_drawn) / self.total_cells
        valid = self._get_valid_actions()
        surround_info = [1.0 if not v else 0.0 for v in valid]

        vector_obs = np.array([dy, dx, progress_ratio, fill_ratio, scale_ratio, *surround_info], dtype=np.float32)

        # 3개의 입력을 딕셔너리로 반환
        return {"local": local_grid_obs, "global": global_grid_obs, "vector": vector_obs}

    def _get_info(self):
        valid = self._get_valid_actions()
        return {'valid_actions': valid, 'is_stuck': not np.any(valid)}

    def _get_valid_actions(self):
        (r, c) = self.pen_pos
        moves = [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]
        valid = np.zeros(4, dtype=bool)
        t_pos = self.nodes.get(self.current_target)
        for i, (nr, nc) in enumerate(moves):
            if not (0 <= nr < self.grid_size and 0 <= nc < self.grid_size): continue
            if (nr, nc) in self.path_drawn: continue
            if (nr, nc) in self.nodes.values():
                if (nr, nc) == t_pos: valid[i] = True
            else: valid[i] = True
        return valid

    def render(self):
        if self.render_mode == 'rgb_array': return self._render_frame()
        elif self.render_mode == 'human':
            self._render_frame()
            pygame.display.flip()
            self.clock.tick(self.metadata["render_fps"])

    def _render_frame(self):
        if self.screen is None: return
        self.screen.fill((255, 255, 255))
        for (r, c) in self.path_drawn:
            pygame.draw.rect(self.screen, (220, 220, 220), (c*self.cell_size, r*self.cell_size, self.cell_size, self.cell_size))
        for num, (r, c) in self.nodes.items():
            center = (c*self.cell_size + self.cell_size//2, r*self.cell_size + self.cell_size//2)
            color = (0, 150, 0) if num == self.current_target else (0, 0, 200)
            pygame.draw.circle(self.screen, color, center, self.cell_size//3)
            if self.font:
                text_surf = self.font.render(str(num), True, (255, 255, 255))
                text_rect = text_surf.get_rect(center=center)
                self.screen.blit(text_surf, text_rect)
        curr_center = (self.pen_pos[1]*self.cell_size + self.cell_size//2, self.pen_pos[0]*self.cell_size + self.cell_size//2)
        pygame.draw.circle(self.screen, (200, 0, 0), curr_center, self.cell_size//4)
        for i in range(self.grid_size + 1):
            pygame.draw.line(self.screen, (100,100,100), (0, i*self.cell_size), (self.grid_size*self.cell_size, i*self.cell_size))
            pygame.draw.line(self.screen, (100,100,100), (i*self.cell_size, 0), (i*self.cell_size, self.grid_size*self.cell_size))
        return np.transpose(pygame.surfarray.array3d(self.screen), (1, 0, 2))

    def close(self):
        if self.screen is not None: pygame.display.quit(); pygame.quit(); self.screen = None

    # --- 맵 생성 로직 (필수) ---
    def _generate_solvable_map(self):
        strategies = ['basic_snake', 'spiral', 'hybrid_vert', 'hybrid_horz']
        strategy = random.choice(strategies)
        if strategy == 'basic_snake': full_path = self._generate_snake_path()
        elif strategy == 'spiral': full_path = self._generate_spiral_path()
        elif strategy == 'hybrid_vert': full_path = self._generate_hybrid_path(mode='vert_first')
        else: full_path = self._generate_hybrid_path(mode='horz_first')
        self.valid_cells_count = len(full_path)
        path_indices = [0]
        available_indices = range(1, len(full_path) - 1)
        needed_middle = self.num_nodes - 2
        safe_k = min(len(available_indices), needed_middle)
        if safe_k > 0:
            middle_indices = sorted(random.sample(available_indices, safe_k))
            path_indices.extend(middle_indices)
        path_indices.append(len(full_path) - 1)
        self.nodes = {}
        for i, idx in enumerate(path_indices):
            self.nodes[i + 1] = full_path[idx]
        self.actual_num_nodes = len(self.nodes)
        self.pen_pos = self.nodes[1]

    def _generate_hybrid_path(self, mode='vert_first'):
        path = []
        if mode == 'vert_first':
            if self.grid_size < 4: return self._generate_snake_path()
            split_col = random.randint(1, self.grid_size - 2)
            for c in range(split_col + 1):
                col_cells = [(r, c) for r in range(self.grid_size)]
                if c % 2 == 1: col_cells.reverse()
                path.extend(col_cells)
            last_r, last_c = path[-1]
            remaining_cols = list(range(split_col + 1, self.grid_size))
            if last_r == self.grid_size - 1: rows = list(range(self.grid_size - 1, -1, -1))
            else: rows = list(range(self.grid_size))
            for i, r in enumerate(rows):
                row_cells = [(r, c) for c in remaining_cols]
                if i % 2 == 1: row_cells.reverse()
                path.extend(row_cells)
        else:
            if self.grid_size < 4: return self._generate_snake_path()
            split_row = random.randint(1, self.grid_size - 2)
            for r in range(split_row + 1):
                row_cells = [(r, c) for c in range(self.grid_size)]
                if r % 2 == 1: row_cells.reverse()
                path.extend(row_cells)
            last_r, last_c = path[-1]
            remaining_rows = list(range(split_row + 1, self.grid_size))
            if last_c == self.grid_size - 1: cols = list(range(self.grid_size - 1, -1, -1))
            else: cols = list(range(self.grid_size))
            for i, c in enumerate(cols):
                col_cells = [(r, c) for r in remaining_rows]
                if i % 2 == 1: col_cells.reverse()
                path.extend(col_cells)
        return path

    def _generate_spiral_path(self):
        path = []; r, c = 0, 0; dr, dc = 0, 1
        visited = np.zeros((self.grid_size, self.grid_size), dtype=bool)
        for _ in range(self.total_cells):
            path.append((r, c)); visited[r, c] = True
            nr, nc = r + dr, c + dc
            if not (0 <= nr < self.grid_size and 0 <= nc < self.grid_size) or visited[nr, nc]:
                dr, dc = dc, -dr; nr, nc = r + dr, c + dc
            if not (0 <= nr < self.grid_size and 0 <= nc < self.grid_size) or visited[nr, nc]: break
            r, c = nr, nc
        return path

    def _generate_snake_path(self):
        path = []
        strategy = random.choice(['horz', 'vert', 'horz_rev', 'vert_rev'])
        if strategy == 'horz':
            for r in range(self.grid_size):
                row_cells = [(r, c) for c in range(self.grid_size)]
                if r % 2 == 1: row_cells.reverse()
                path.extend(row_cells)
        elif strategy == 'vert':
            for c in range(self.grid_size):
                col_cells = [(r, c) for r in range(self.grid_size)]
                if c % 2 == 1: col_cells.reverse()
                path.extend(col_cells)
        elif strategy == 'horz_rev':
            for r in range(self.grid_size - 1, -1, -1):
                row_cells = [(r, c) for c in range(self.grid_size)]
                if (self.grid_size - 1 - r) % 2 == 1: row_cells.reverse()
                path.extend(row_cells)
        elif strategy == 'vert_rev':
            for c in range(self.grid_size - 1, -1, -1):
                col_cells = [(r, c) for r in range(self.grid_size)]
                if (self.grid_size - 1 - c) % 2 == 1: col_cells.reverse()
                path.extend(col_cells)
        return path

In [5]:
# 💡 이 코드는 Global View와 Local View를 동시에 처리하는 3-Stream 구조입니다.

# =========================================================
# 1. 커스텀 레이어 정의 (Agent 클래스보다 반드시 먼저 정의되어야 함!)
# =========================================================
@tf.keras.utils.register_keras_serializable()
class DuelingMergeLayer(layers.Layer):
    """
    [듀얼링 병합 레이어]
    역할: V(상태 가치)와 A(행동 이득)를 합쳐 최종 Q값을 만듭니다.
    이 수식이 모델의 안정성을 크게 높여줍니다.
    """
    def __init__(self, **kwargs):
        super(DuelingMergeLayer, self).__init__(**kwargs)

    def call(self, inputs):
        value, advantage = inputs
        # Q = V + (A - mean(A)) 공식 적용
        return value + (advantage - tf.reduce_mean(advantage, axis=1, keepdims=True))

    def compute_output_shape(self, input_shape):
        # 출력 모양은 행동 개수만큼의 Q값 벡터입니다.
        return input_shape[1]

    def get_config(self):
        return super(DuelingMergeLayer, self).get_config()

# =========================================================
# 2. Dual View 에이전트 클래스 정의
# =========================================================
class DualDQNAgent:
    """
    [3-Stream 아키텍처]
    Local CNN (전술) + Global CNN (전략) + Vector MLP (상황 인식)
    """
    def __init__(self, local_shape, global_shape, vector_shape, action_size):
        # 입력 쉐이프 3개 저장 (Env에서 받은 크기)
        self.local_shape = local_shape
        self.global_shape = global_shape
        self.vector_shape = vector_shape
        self.action_size = action_size

        self.buffer = deque(maxlen=50000) # 경험 리플레이 버퍼
        self.gamma = 0.99 # 할인율 (미래 보상 중요도)
        self.epsilon = 1.0 # 초기 탐험률
        self.epsilon_min = 0.01
        self.learning_rate = 0.00025 # 안정적 학습을 위한 낮은 학습률

        # 메인 모델과 타겟 모델(Double DQN) 생성
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        """신경망 구조 설계 (3개의 독립적인 경로)"""

        # --- [1] Local View (7x7) 처리: 미세 컨트롤 ---
        # 맵이 커져도 이 시야는 변하지 않습니다. (범용성의 핵심)
        input_local = layers.Input(shape=self.local_shape, name='local_input')
        x1 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_local)
        x1 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x1)
        x1 = layers.Flatten()(x1) # 1차원 벡터로 변환

        # --- [2] Global View (12x12) 처리: 전역 전략 (Global Eyes) ---
        # 맵 전체의 연결성, 큰 흐름을 파악하여 '길 끊어먹는 실수'를 방지합니다.
        input_global = layers.Input(shape=self.global_shape, name='global_input')
        x2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_global)
        x2 = layers.MaxPooling2D((2, 2))(x2) # 크기를 줄여서(Pooling) 추상화 및 중요한 패턴만 추출
        x2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x2)
        x2 = layers.Flatten()(x2) # 1차원 벡터로 변환

        # --- [3] Vector Info 처리: 나침반 및 상태 ---
        # 맵 크기 비율, 목표까지의 방향 등 수치 정보를 처리합니다.
        input_vector = layers.Input(shape=self.vector_shape, name='vector_input')
        y = layers.Dense(64, activation='relu')(input_vector)

        # --- [4] 통합 (Merge) ---
        # 세 가지 정보를 합쳐서 최종적인 판단 근거를 만듭니다.
        combined = layers.Concatenate()([x1, x2, y])
        z = layers.Dense(256, activation='relu')(combined) # 은닉층의 용량 증가

        # --- [5] Dueling Output (최종 결정) ---
        value = layers.Dense(1, activation='linear', name='value')(z) # V: 이 상태의 가치
        advantage = layers.Dense(self.action_size, activation='linear', name='advantage')(z) # A: 각 행동의 이득

        # DuelingMergeLayer를 사용해 최종 Q값을 계산
        output = DuelingMergeLayer(name='q_values')([value, advantage])

        # 모델 조립 및 컴파일
        model = models.Model(inputs=[input_local, input_global, input_vector], outputs=output)
        model.compile(loss=losses.Huber(), optimizer=optimizers.Adam(learning_rate=self.learning_rate, clipnorm=1.0))
        return model

    def update_target_model(self):
        """타겟 모델을 메인 모델의 최신 가중치로 동기화합니다."""
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done, next_valid_actions):
        """경험을 버퍼에 저장합니다."""
        self.buffer.append((state, action, reward, next_state, done, next_valid_actions))

    @tf.function
    def _predict(self, local, global_, vector, model_type='main'):
        """컴파일된 예측 함수 (추론 속도 최적화)"""
        if model_type == 'target':
            return self.target_model([local, global_, vector], training=False)
        else:
            return self.model([local, global_, vector], training=False)

    def act(self, state, valid_actions):
        """행동 선택 (Epsilon-Greedy + Masking)"""
        # 1. 탐험 (랜덤 선택)
        if np.random.rand() <= self.epsilon:
            valid_indices = np.where(valid_actions)[0]
            if len(valid_indices) == 0: return 0
            return np.random.choice(valid_indices)

        # 2. 활용 (모델 추론)
        # 3개의 입력을 배치 크기 1로 변환
        local_in = tf.expand_dims(tf.convert_to_tensor(state['local'], dtype=tf.float32), 0)
        global_in = tf.expand_dims(tf.convert_to_tensor(state['global'], dtype=tf.float32), 0)
        vector_in = tf.expand_dims(tf.convert_to_tensor(state['vector'], dtype=tf.float32), 0)

        # 예측 수행
        q_values = self._predict(local_in, global_in, vector_in, 'main').numpy()[0]

        # 마스킹: 갈 수 없는 곳(-np.inf)을 제외하고 최고 Q값 선택
        q_values[~valid_actions] = -np.inf
        return np.argmax(q_values)

    def replay(self, batch_size):
        """경험 리플레이를 통한 학습"""
        if len(self.buffer) < batch_size: return
        minibatch = random.sample(self.buffer, batch_size)

        # 데이터 분리 및 배열화
        locals_ = np.array([d[0]['local'] for d in minibatch], dtype=np.float32)
        globals_ = np.array([d[0]['global'] for d in minibatch], dtype=np.float32)
        vectors = np.array([d[0]['vector'] for d in minibatch], dtype=np.float32)
        # ... (나머지 데이터 분리)
        actions = np.array([d[1] for d in minibatch])
        rewards = np.array([d[2] for d in minibatch], dtype=np.float32)
        next_locals = np.array([d[3]['local'] for d in minibatch], dtype=np.float32)
        next_globals = np.array([d[3]['global'] for d in minibatch], dtype=np.float32)
        next_vectors = np.array([d[3]['vector'] for d in minibatch], dtype=np.float32)
        dones = np.array([d[4] for d in minibatch], dtype=np.float32)
        next_valids = np.array([d[5] for d in minibatch], dtype=bool)

        # Double DQN 정답지 계산
        next_q_target = self._predict(next_locals, next_globals, next_vectors, 'target').numpy()
        next_q_main = self._predict(next_locals, next_globals, next_vectors, 'main').numpy()

        # Bellman Equation 적용
        next_q_main[~next_valids] = -np.inf
        best_actions = np.argmax(next_q_main, axis=1)

        row_indices = np.arange(batch_size)
        next_q_values = next_q_target[row_indices, best_actions]
        targets = rewards + self.gamma * next_q_values * (1 - dones)

        # 모델 학습
        target_f = self._predict(locals_, globals_, vectors, 'main').numpy()
        target_f[row_indices, actions] = targets

        self.model.train_on_batch([locals_, globals_, vectors], target_f)

    def reheat_epsilon(self, target_epsilon=0.6, force=False):
        """엡실론 재가열 (슬럼프 탈출용)"""
        if force or (self.epsilon < target_epsilon):
            print(f"   >>> 🔥 [재가열] Epsilon: {self.epsilon:.2f} -> {target_epsilon:.2f}")
            self.epsilon = target_epsilon

In [6]:
# =========================================================
# [수정된] 성능 검증 실행 로직
# =========================================================
if __name__ == "__main__":
    # 1. 모델 경로 설정 (사용자 환경에 맞게)
    GDRIVE_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/reinforcement learning/Final_Model_4to8'

    # 체크포인트 모델이 더 최신일 확률이 높으므로 우선 로드
    # MODEL_PATH = os.path.join(GDRIVE_BASE_PATH, "zip_checkpoint.keras")
    # 만약 졸업 모델을 쓰고 싶다면 아래 주석 해제
    MODEL_PATH = os.path.join(GDRIVE_BASE_PATH, "zip_universal_model.keras")

    print(f"📂 모델 로드 중: {MODEL_PATH}")

    if not os.path.exists(MODEL_PATH):
        print("❌ 모델 파일이 없습니다. 학습을 먼저 완료해주세요.")
        agent_model = None
    else:
        try:
            # 커스텀 객체 명시 필수
            agent_model = tf.keras.models.load_model(
                MODEL_PATH,
                custom_objects={'DuelingMergeLayer': DuelingMergeLayer},
                compile=False
            )
            print("✅ 모델 로드 성공!")
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            agent_model = None

    if agent_model:
        # 테스트 범위
        test_sizes = list(range(4, 13)) # 4x4 ~ 12x12
        EPISODES_PER_SIZE = 100

        # 테스트용 환경 (학습 때 썼던 ZipGeneralEnv 클래스 그대로 사용)
        test_env = ZipGeneralEnv(max_grid_size=12, render_mode=None)

        print("\n🧪 [성능 검증 시작] 4x4 ~ 12x12 (Epsilon=0.0, Valid Masking ON)")
        print("-" * 65)
        print(f"{'Grid Size':<10} | {'Success Rate':<15} | {'Avg Score':<15} | {'Status'}")
        print("-" * 65)

        for size in test_sizes:
            test_env.set_curriculum_level(size) # 해당 크기 고정
            success_count = 0
            scores = []

            for ep in range(EPISODES_PER_SIZE):
                # 1. 초기화 및 첫 번째 Valid Actions 획득
                obs, info = test_env.reset()
                valid_actions = info['valid_actions'] # ⭐️ [핵심] 마스크 가져오기

                done = False
                total_reward = 0

                while not done:
                    # 2. 모델 예측을 위한 차원 확장
                    local_in = tf.expand_dims(obs['local'], 0)
                    global_in = tf.expand_dims(obs['global'], 0)
                    vector_in = tf.expand_dims(obs['vector'], 0)

                    # 3. Q값 예측
                    q_values = agent_model([local_in, global_in, vector_in], training=False).numpy()[0]

                    # 4. ⭐️ [핵심] 마스킹 적용 (학습 때와 동일한 조건)
                    # 갈 수 없는 곳(valid=0)은 -무한대 점수로 만들어 선택 불가하게 함
                    q_values[valid_actions == 0] = -1e9

                    # 5. 최적 행동 선택
                    action = np.argmax(q_values)

                    # 6. 환경 진행
                    obs, reward, terminated, truncated, info = test_env.step(action)

                    # ⭐️ 다음 스텝을 위한 마스크 갱신
                    valid_actions = info['valid_actions']

                    done = terminated or truncated
                    total_reward += reward

                scores.append(total_reward)

                # 성공 기준 (잭팟 점수 근처)
                target_score = 500.0 + (size * size)
                if total_reward >= target_score * 0.9:
                    success_count += 1

            success_rate = (success_count / EPISODES_PER_SIZE) * 100
            avg_score = np.mean(scores)

            status = "🟢 Great" if success_rate >= 90 else "🟡 Good" if success_rate >= 50 else "🔴 Fail"
            print(f"{size}x{size:<8} | {success_rate:6.1f}%        | {avg_score:8.1f}        | {status}")

        print("-" * 65)
        print("✅ 검증 완료.")

📂 모델 로드 중: /content/drive/MyDrive/Colab Notebooks/reinforcement learning/Final_Model_4to8/zip_universal_model.keras
✅ 모델 로드 성공!

🧪 [성능 검증 시작] 4x4 ~ 12x12 (Epsilon=0.0, Valid Masking ON)
-----------------------------------------------------------------
Grid Size  | Success Rate    | Avg Score       | Status
-----------------------------------------------------------------
4x4        |   99.0%        |    551.0        | 🟢 Great
5x5        |   97.0%        |    557.3        | 🟢 Great
6x6        |   87.0%        |    537.1        | 🟡 Good
7x7        |   84.0%        |    549.8        | 🟡 Good
8x8        |   84.0%        |    588.7        | 🟡 Good
9x9        |   54.0%        |    417.1        | 🟡 Good
10x10       |   26.0%        |    301.3        | 🔴 Fail
11x11       |   11.0%        |    255.3        | 🔴 Fail
12x12       |   25.0%        |    316.6        | 🔴 Fail
-----------------------------------------------------------------
✅ 검증 완료.
